In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-134939")
exp = Experiment(workspace=ws, name="quick-starts-ws-134939_exp2")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RDU5S6TV6 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-134939
Azure region: southcentralus
Subscription id: 610d6e37-4747-4a20-80eb-3aad70a55f43
Resource group: aml-quickstarts-134939


In [2]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
cpu_cluster_name = 'ampproj1compute'
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal, choice
import os
from azureml.core import Environment
from azureml.core import ScriptRunConfig


# Specify parameter sampler
ps = RandomParameterSampling( 
    {
        '--C' : choice(1,10,20,50,100,200,500,1000),
        '--max_iter': choice(50,100,200,300)
    }
)
# Specify a Policy
# Specify parameter sampler
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1, delay_evaluation=5)


if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# est = ### YOUR CODE HERE ###
sklearn_env = Environment.get(workspace=ws, name='AzureML-Tutorial')
# src = ScriptRunConfig(source_directory='.', script='train.py', arguments=['--kernel', 'linear', '--penalty', 1.0], environment=sklearn_env)
src = ScriptRunConfig(source_directory='.', script='train.py', environment=sklearn_env)
src.run_config.target = compute_target
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=100,
    max_concurrent_runs=4)
print(hyperdrive_config)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
from azureml.core import Experiment, Workspace

# run = Experiment(ws,'train2').submit(src)
# run.wait_for_completion(show_output=True)

experiment = Experiment(ws, exp.name)
hyperdrive_run = experiment.submit(hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)
RunDetails(hyperdrive_run).show()

RunId: HD_644fed66-4541-4489-bdb8-229bb589affd
Web View: https://ml.azure.com/experiments/quick-starts-ws-134939_exp2/runs/HD_644fed66-4541-4489-bdb8-229bb589affd?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-134939/workspaces/quick-starts-ws-134939

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-16T19:49:21.204417][API][INFO]Experiment created<END>\n""<START>[2021-01-16T19:49:21.881016][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2021-01-16T19:49:22.0482620Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-01-16T19:49:22.193716][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_644fed66-4541-4489-bdb8-229bb589affd
Web View: https://ml.azure.com/experiments/quick-starts-ws-134939_exp2/runs/HD_644fed66-4541-4489-bdb8-229bb589affd?w

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [5]:
import joblib
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

for child_run in hyperdrive_run.get_children_sorted_by_primary_metric():
    print(child_run)

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details() ['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print(' -Accuracy:', best_run_metrics['Accuracy'])
print(' -Regularization Rate:',parameter_values)

# Register model
best_run.register_model(model_path='outputs/amptrain_1.pkl', model_name='amptrain_1',
                        tags={'Training context':'Hyperdrive'},
                        properties={'Accuracy': best_run_metrics['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

{'run_id': 'HD_644fed66-4541-4489-bdb8-229bb589affd_39', 'hyperparameters': '{"--C": 1000, "--max_iter": 300}', 'best_primary_metric': 0.9089529590288316, 'status': 'Completed'}
{'run_id': 'HD_644fed66-4541-4489-bdb8-229bb589affd_14', 'hyperparameters': '{"--C": 0.1, "--max_iter": 200}', 'best_primary_metric': 0.908649468892261, 'status': 'Completed'}
{'run_id': 'HD_644fed66-4541-4489-bdb8-229bb589affd_31', 'hyperparameters': '{"--C": 100, "--max_iter": 200}', 'best_primary_metric': 0.9084673748103187, 'status': 'Completed'}
{'run_id': 'HD_644fed66-4541-4489-bdb8-229bb589affd_1', 'hyperparameters': '{"--C": 1000, "--max_iter": 200}', 'best_primary_metric': 0.9084066767830046, 'status': 'Completed'}
{'run_id': 'HD_644fed66-4541-4489-bdb8-229bb589affd_11', 'hyperparameters': '{"--C": 1, "--max_iter": 300}', 'best_primary_metric': 0.9083459787556905, 'status': 'Completed'}
{'run_id': 'HD_644fed66-4541-4489-bdb8-229bb589affd_8', 'hyperparameters': '{"--C": 20, "--max_iter": 200}', 'best_pr

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_url_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data_url_path)

In [8]:

import clean_data
from azureml.core import Dataset

# Use the clean_data function to clean your data.
x, y = clean_data.clean_data(ds)
# train_data, test_data = ds.random_split(percentage=0.5, seed=50)
dsnew = x.join(y)
dsnew.to_csv('dataset_automl.csv')


In [ ]:
ds_automl = Dataset.get_by_name(workspace=ws, name="ds_automl", version='latest')
train_data, test_data = ds_automl.random_split(percentage=0.5, seed=50)

In [10]:

from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target=compute_target,
    experiment_timeout_minutes=15,
    task='classification',
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name="y",
    n_cross_validations=3
)


In [11]:
# Submit your automl run

### YOUR CODE HERE ###

from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

experiment = Experiment(ws, "automl_test_experiment")
run_auto = experiment.submit(config=automl_config, show_output=True)

RunDetails(run_auto).show()


Running on remote.
No run_configuration provided, running on ampproj1compute with default configuration
Running on remote compute: ampproj1compute
Parent Run ID: AutoML_0cbfc867-f0f5-4b66-aca6-db2ebdadeb59

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias t

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [12]:
best_run, fitted_model = run_auto.get_output()

print(best_run)
print(fitted_model)


Run(Experiment: automl_test_experiment,
Id: AutoML_0cbfc867-f0f5-4b66-aca6-db2ebdadeb59_3,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    max_samples=None,
                                                                                                    min_impurity_decrease=0.0,
            

In [14]:
compute_target.delete()